In [1]:
import os

import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import load_model, Model, Sequential

ImportError: numpy.core.multiarray failed to import

ImportError: numpy.core.multiarray failed to import

In [2]:
path = './models/autoencoder_0.h5'
autoencoder = load_model(path)

NameError: name 'load_model' is not defined

In [27]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 64)          0         
__________

In [28]:
for layer in autoencoder.layers:
    layer.trainable = False

In [29]:
del autoencoder.layers[-9:]
#for _ in range(9):
#    autoencoder.layers.pop()

autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 64)          0         
__________

In [30]:
encoder = autoencoder.layers[-1].output

In [31]:
num_classes = 10

model = Sequential(autoencoder.layers)

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(rate = 0.5))
model.add(Dense(num_classes, activation='softmax'))

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 128)         73856     
__________

In [33]:
model.compile(optimizer = 'Adam', loss = 'mse', metrics=['accuracy'])

In [43]:
def load_dataset(img_width = 32, img_heigth = 32):
    path = "data/"
    
    folders = os.listdir(path)

    X = []
    Y = []

    for folder in folders:

        if os.path.isdir(path + folder):
            frames = os.listdir(path + folder)
            print("Loading: " + folder)

            for frame in frames:
                # load a new frame
                new_frame = cv2.imread(path + folder + "/" + frame)

                # convert to grayscale
                new_frame = cv2.cvtColor(new_frame, cv2.COLOR_RGB2GRAY)
                # resize frame
                new_frame = cv2.resize(new_frame, (img_width, img_heigth))

                X.append(new_frame.astype("float32")/255.)
                Y.append(folder)

    X = np.array(X).reshape((-1, img_width, img_heigth, 1))
    Y = pd.get_dummies(Y).values

    return X, Y


In [44]:
X, Y = load_dataset()
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 23)

Loading: jump_subtracted
Loading: skip_subtracted
Loading: wave1_subtracted
Loading: run_subtracted
Loading: wave2_subtracted
Loading: jack_subtracted
Loading: walk_subtracted
Loading: bend_subtracted
Loading: pjump_subtracted
Loading: side_subtracted


In [46]:
model.fit(x_train, y_train,
        batch_size=1,
        epochs=10,
        verbose=1,
        validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 3915 samples, validate on 1679 samples
Epoch 1/10
3915/3915 [==============================] - 6s 1ms/step - loss: 0.0716 - acc: 0.4130 - val_loss: 0.0573 - val_acc: 0.5337
Epoch 2/10
3915/3915 [==============================] - 5s 1ms/step - loss: 0.0590 - acc: 0.5172 - val_loss: 0.0514 - val_acc: 0.5902
Epoch 3/10
3915/3915 [==============================] - 5s 1ms/step - loss: 0.0547 - acc: 0.5550 - val_loss: 0.0504 - val_acc: 0.5896
Epoch 4/10
3915/3915 [==============================] - 5s 1ms/step - loss: 0.0534 - acc: 0.5693 - val_loss: 0.0502 - val_acc: 0.5890
Epoch 5/10
3915/3915 [==============================] - 5s 1ms/step - loss: 0.0511 - acc: 0.5773 - val_loss: 0.0482 - val_acc: 0.6045
Epoch 6/10
3915/3915 [==============================] - 5s 1ms/step - loss: 0.0510 - acc: 0.5801 - val_loss: 0.0473 - val_acc: 0.6182
Epoch 7/10
3915/3915 [==============================] - 5s 1ms/step - loss: 0.0494 - acc: 0.6005 - v